In [1]:
import cmipld 
from cmipld.utils.server_tools.offline import LD_server
from pyld import jsonld

ldpath = cmipld.utils.io.ldpath() 

repo_url = cmipld.utils.git.url()
io_url = cmipld.utils.git.url2io(repo_url)
whoami = cmipld.reverse_mapping()[io_url]

localserver = LD_server( copy=[[ldpath, whoami]], override='y')
localhost = localserver.start_server(8084)

ld = jsonld.expand('cmip7:experiment/1pctco2',options={'extractAllScripts':  True})


# print(help(cmipld.jsonld.expand))

make test for url2io
Copying the repo into LocalServer  /Users/daniel.ellis/WIPwork/CMIP7_CVs/src-data/ --> cmip7
Repositories copied into <TemporaryDirectory '/var/folders/hc/s_7lggq12nndglbdyrn3f91m1l58yd/T/cmipld_local_upn1op0k2025-05-01T01:51:26'>


-----


Created SSL certificates in: 
/var/folders/hc/s_7lggq12nndglbdyrn3f91m1l58yd/T/cmipld_local_upn1op0k2025-05-01T01:51:26

╭─────────────────────────────────────────────── Prefix Resolvers ────────────────────────────────────────────────╮
│ cf: https://wcrp-cmip.github.io/CF/                                                                             │
│ cmip7: https://wcrp-cmip.github.io/CMIP7-CVs/                                                                   │
│ obs4mips: https://wolfiex.github.io/obs4MIPs-cmor-tables-ld/                                                    │
│ universal: https://wcrp-cmip.github.io/WCRP-universe/                                                           │
│ variables: https://wcrp-cmip.github.io/MIP-variables/                                                           │
│ cmip6plus: https://wcrp-cmip.github.io/CMIP6Plus_CVs/                                                           │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

                                        Regex-Based URL Redirects (by Host)                                        
┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃                        Host                        ┃ Rules                                                      ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
└────────────────────────────────────────────────────┴────────────────────────────────────────────────────────────┘

Serving /var/folders/hc/s_7lggq12nndglbdyrn3f91m1l58yd/T/cmipld_local_upn1op0k2025-05-01T01:51:26 at 
https://localhost:8084

add the mappings here to the server


In [3]:
cmipld.utils.get("cmip7:experiment/1pctco2",depth=10,compact=False)

AttributeError: 'list' object has no attribute 'items'

In [ ]:
from pyld import jsonld


def get(link, compact=True, depth=10):
    """
    Retrieve and resolve a JSON-LD document from a URL.

    Args:
        link (str): URL of the JSON-LD document.
        compact (bool): Whether to compact the final document using the original context.
        depth (int): Maximum recursion depth for resolving @id references.

    Returns:
        dict or list: The fully resolved JSON-LD document.
    """
    expanded = jsonld.expand(link, options={'extractAllScripts': True})
    resolved = _resolve_ids(expanded, compact=compact, depth=depth)

    if compact:
        return jsonld.compact(resolved, link)

    return resolved


def _resolve_ids(data, compact=True, depth=2):
    """
    Recursively resolve @id references in a JSON-LD structure.

    Args:
        data (dict or list): The expanded JSON-LD data to process.
        compact (bool): Whether to compact merged results when combining data.
        depth (int): Remaining depth for recursive resolution.

    Returns:
        dict or list: JSON-LD data with resolved references.
    """
    if depth <= 0:
        return data

    if isinstance(data, dict):
        if '@id' in data and '@type' not in data and data['@id'].startswith('http'):
            try:
                # Fetch and resolve linked document
                expanded_ref = get(data['@id'], compact=compact, depth=depth - 1)
            except jsonld.JsonLdError:
                print('\nWARNING: missing @id:', data['@id'])
                expanded_ref = None

            if expanded_ref:
                if len(data) > 1:
                    # Merge current data (minus @id) with resolved reference
                    data = {**expanded_ref, **{k: v for k, v in data.items() if k != '@id'}}
                    data = jsonld.compact(data, expanded_ref)
                else:
                    data = expanded_ref

        return {key: _resolve_ids(value, compact, depth) for key, value in data.items()}

    if isinstance(data, list):
        return [_resolve_ids(item, compact, depth) for item in data]

    return data  # Base case for primitive values


In [ ]:
out = get('cmip7:experiment/scen7-vlhoc',compact=True, depth=88)

print(json.dumps(out,indent=8))

In [ ]:
# rid = _resolve_ids(ld, compact=True, depth=10)

In [ ]:
# jsonld.compact(rid,{'@context': 'cmip7:experiment/1pctco2',
#  'id': 'cmip7:experiment/1pctco2',
#  'type': ['wcrp:experiment', 'cmip7']})

In [ ]:
# import json
# print(json.dumps(ld,indent=9))

In [ ]:
jsonld.expand('cmip7:experiment/scen7-vlhoc')